In [50]:
import os
import requests

import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

%cd ..
from model.BasicModel import BasicModel

# Load environment variables
load_dotenv()
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")


C:\


In [4]:
df_path = os.path.join(os.getcwd(), 'data', 'data.csv')
df = pd.read_csv(df_path)
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

In [7]:
df_train = df[['acousticness', 'danceability','energy',
               'instrumentalness', 'key', 'liveness', 'loudness',
               'mode','speechiness', 'tempo',
               'valence']]

In [8]:
df_train.dtypes

acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [9]:
df_train.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.995,0.708,0.1950,0.563,10,0.1510,-12.428,1,0.0506,118.469,0.7790
1,0.994,0.379,0.0135,0.901,8,0.0763,-28.454,1,0.0462,83.972,0.0767
2,0.604,0.749,0.2200,0.000,5,0.1190,-19.924,0,0.9290,107.177,0.8800
3,0.995,0.781,0.1300,0.887,1,0.1110,-14.734,0,0.0926,108.003,0.7200
4,0.990,0.210,0.2040,0.908,11,0.0980,-16.829,1,0.0424,62.149,0.0693


In [51]:
model = BasicModel()

In [52]:
model.build(10)

In [53]:
model.fit(df_train)

NearestNeighbors(n_neighbors=10)

In [57]:
test = test.reshape(1, -1)
scores, indices = model.predict(test)

In [63]:
results = []
for i, index in enumerate(indices[0]):
    track_id = df.iloc[index]['id']
    artists = df.iloc[index]['artists']
    title = df.iloc[index]['name']
    results.append({'index': index,
                    'track_id': track_id,
                    'artists': artists,
                    'title': title,
                    'score': scores[0][i]})
    
result_table = pd.DataFrame(results)
result_table.sort_values(by='score')

,index,track_id,artists,title,score
0,200,4gZXSffISQgwcUg3FfU6Be,"['Sinclair Lewis', 'Frank Arnold']","Das ist bei uns nicht möglich, Kapitel 137",0.000000
1,48602,1pQXveLHUGAMsug2wHRthv,['Эрнест Хемингуэй'],"Часть 54.3 - За рекой, в тени деревьев",0.787197
2,56475,2ofIYVjot0JJx6KKsPgOzo,['Patricia St. John'],Kapitel 49 - Spuren im Schnee,0.888057
3,210,4jJ1Xp7ccRdH2U3TcylvFI,"['Sinclair Lewis', 'Frank Arnold']","Das ist bei uns nicht möglich, Kapitel 193",1.051426
4,80375,0K9pzedgq0hbPGrYtmvXrB,"['Dale Carnegie', 'Till Hagen', 'Stefan Kamins...","Sorge dich nicht - lebe! - Die Kunst, zu einem...",1.220795
5,63771,2qP0XXkMDFXLryHzPCIsBh,['Patricia St. John'],Kapitel 125 - Spuren im Schnee,1.233951
6,128510,0CYace8npBSwLiEf9Ukr5f,['Эрнест Хемингуэй'],"Часть 31.3 - За рекой, в тени деревьев",1.310362
7,107520,1ltfoW4OvBWS1xlm5OT4cP,['Seweryn Goszczyński'],Chapter 1.9 - Zamek kaniowski,1.344175
8,146185,0dUmRtCY5pYQeIQ9RULRGE,['Patricia St. John'],Kapitel 18 - Spuren im Schnee,1.352645
9,33506,1pqCy035MX2cv4z2eFwEBn,['George Bernard Shaw'],"The Class War, The Four-Hour Day",1.430624
